# Computing wasserstein distance between lesion SUV

## Setup

### Importing base libraries

In [9]:
import os, sys

from tqdm.notebook import tqdm
from typing import List, Dict

import pandas as pd
import numpy as np
import networkx as nx

import plotly.express as px
import plotly.io as pio

pio.templates.default = 'seaborn'

In [10]:
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src'))

### Fetch data

In [11]:
CONNECTION_DIR = '/Volumes/lts4-immuno/'

fpath = os.path.join(CONNECTION_DIR, 'data_2021-11-01', 'LesionSUVs')

filenames = os.listdir(fpath)

In [12]:
def extract_gpcr_id(filename: str):
    # Extract the GPCR_ID from the filename
    # Filenames follow the following naming structure
    # `sub-${GPCR_ID}_lesion-suv.hdf`
    name, ext = filename.split('.')
    _, desc, _ = name.split('-')
    gpcr_id, _ = desc.split('_')
    
    return int(gpcr_id)

## Save Wasserstein distances to file

In [13]:
from math import comb
from itertools import combinations
from scipy.stats import wasserstein_distance

# Create an empty DataFrame from file with header
storage = pd.read_csv('wasserstein-distance.csv')

# Iterate over all filenames to extract post-1 wasserstein distance for all patients
pbar = tqdm(zip(map(extract_gpcr_id, filenames), filenames), total=len(filenames))
for gpcr_id, filename in pbar:
    pbar.set_description(f'Processing patient {gpcr_id}')
    
    # Open connection to HDF file
    store = pd.HDFStore(os.path.join(fpath, filename))

    for study in ['pre_01', 'post_01']:
    
        # Make sure the file contains a post-1 study
        if store.__contains__(study):
            
            df = store.get(study)

            # Iterate over all combinations of lesions
            lesions_pbar = tqdm(combinations(df.columns, 2), total=comb(len(df.columns), 2), leave=False)
            for i, j in lesions_pbar:
                lesions_pbar.set_description(f'Computing Wasserstein distance for lesions ({i} -- {j})')
                
                # Fetch data for each lesion and remove NaN values
                x, y = df[i].to_numpy(), df[j].to_numpy()
                x, y = x[~np.isnan(x)], y[~np.isnan(y)]
                # Compute wasserstein distance between recordings
                d = wasserstein_distance(x, y)
                # Append row to DataFrame
                storage = storage.append(
                    dict(gpcr_id=gpcr_id, study_name=study, lesion_i=i, lesion_j=j, wasserstein_distance=d),
                    ignore_index=True)

  0%|          | 0/130 [00:00<?, ?it/s]

  0%|          | 0/3741 [00:00<?, ?it/s]

  0%|          | 0/210 [00:00<?, ?it/s]

  0%|          | 0/3240 [00:00<?, ?it/s]

  0%|          | 0/4950 [00:00<?, ?it/s]

...

  0%|          | 0/465 [00:00<?, ?it/s]

  0%|          | 0/1596 [00:00<?, ?it/s]

  0%|          | 0/3403 [00:00<?, ?it/s]

  0%|          | 0/2850 [00:00<?, ?it/s]

  0%|          | 0/3741 [00:00<?, ?it/s]

In [14]:
storage.to_csv('wasserstein-distance.csv', index=False)

## Extract radiomics information

In [8]:
# Create an empty DataFrame from file with header
lesions = pd.read_csv('lesions.csv')

# Iterate over all filenames
pbar = tqdm(zip(map(extract_gpcr_id, filenames), filenames), total=len(filenames))
for gpcr_id, filename in pbar:
    pbar.set_description(f'Processing patient {gpcr_id}')
    
    # Open connection to HDF file
    store = pd.HDFStore(os.path.join(fpath, filename))
    studies = store.keys()
    
    # Iterate over all combinations of lesions
    for study in studies:
        
        # Fetch data
        df = store.get(study)
        
        for lesion in df.columns:
            
            x = df[lesion].to_numpy()
            x = x[~np.isnan(x)]
            
            voxels = x.shape[0]
            min_suv_val = x.min()
            mean_suv_val = x.mean()
            median_suv_val = np.median(x)
            sd_suv_val = x.std()
            max_suv_val = x.max()
        
            # Append row to DataFrame
            lesions = lesions.append(
                dict(gpcr_id=gpcr_id, study_name=study, lesion_label_id=lesion, voxels=voxels,
                     mean_suv_val=mean_suv_val, max_suv_val=max_suv_val, min_suv_val=min_suv_val,
                     median_suv_val=median_suv_val, sd_suv_val=sd_suv_val), ignore_index=True)

  0%|          | 0/130 [00:00<?, ?it/s]

In [10]:
lesions.to_csv('lesions.csv', index=False)